# Creation of a pandapipes Heating Network

There are several aspects to consider when constructing a heating network. To get a basic introduction to creating a pandapipes network, please see the tutorial ["Creating pandapipes Networks"](https://github.com/e2nIEE/pandapipes/blob/master/tutorials/creating_a_simple_network.ipynb) first. For the pipes, additional parameters must be specified: The heat transfer coefficient (*alpha_w_per_m2k*), which determines how well or poorly the pipe insulates and the number of internal pipe sections (*sections*). Likewise the ambient temperature of the pipe (*text_k*) can be changed, which is 293 K by default. In this case the variable *ambient_temperature* in the [*pipeflow* function](https://pandapipes.readthedocs.io/en/latest/pipeflow.html) should be set to the same value. In addition, for an external grid the variable *type* should be set to "pt" or "t" and a constant temperature value for *t_k* should be defined. Furthermore, start values for the temperatures at the junctions (*tfluid_k*) should be specified.

Please note that only incompressible media can be used for the heating network calculation and the [*mode*](https://pandapipes.readthedocs.io/en/latest/pipeflow/calculation_modes.html#temperature-calculations-pipeflow-option-mode-all-or-mode-heat) in the *pipeflow* function has to be set to "all" or "heat". In case *mode* equals "heat", the user must manually specify a solution vector for the hydraulic calculations. It should also be noted that the temperature calculations are currently still sequential. This means that the calculated temperature values do not influence the hydraulic properties of the medium. Therefore, the calculations are only valid if the properties are not very temperature-dependent or if there are minor changes in temperature.
                                                                                                            
In the following a simple example for the creation and calculation of a network is presented. Here water is used as fluid and the mode "all" is selected.

In [ ]:
import pandapipes
from pandapipes.component_models import Pipe

# create empty network
net = pandapipes.create_empty_network("net", add_stdtypes=False)

# create fluid
pandapipes.create_fluid_from_lib(net, "water", overwrite=True)

# create junctions
junction1 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=290, name="Junction 1", geodata=(0, 0))
junction2 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=290, name="Junction 2", geodata=(2, 0))
junction3 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=290, name="Junction 3", geodata=(4, 0))
junction4 = pandapipes.create_junction(net, pn_bar=3, tfluid_k=290, name="Junction 4", geodata=(2, 2))

# create external grid
pandapipes.create_ext_grid(net, junction=junction1, p_bar=6, t_k=363.15, name="External Grid", type="pt")

# creat sinks
pandapipes.create_sink(net, junction=junction3, mdot_kg_per_s=1, name="Sink 1")
pandapipes.create_sink(net, junction=junction4, mdot_kg_per_s=2, name="Sink 2")

# create pipes
pandapipes.create_pipe_from_parameters(net, from_junction=junction1, to_junction=junction2, length_km=0.1,
                                       diameter_m=0.075, k_mm=0.025, sections=5, alpha_w_per_m2k=100,
                                       text_k=298.15, name="Pipe 1", geodata=[(0, 0), (2, 0)])

pandapipes.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction3, length_km=2,
                                       diameter_m=0.05, k_mm=0.025, sections=4, alpha_w_per_m2k=100,
                                       text_k=298.15, name="Pipe 2", geodata=[(2, 0), (4, 0)])

pandapipes.create_pipe_from_parameters(net, from_junction=junction2, to_junction=junction4, length_km=1,
                                       diameter_m=0.1, k_mm=0.025, sections=8, alpha_w_per_m2k=50,
                                       text_k=298.15, name="Pipe 3", geodata=[(2, 0), (2, 2)])

# run pipeflow
pandapipes.pipeflow(net, stop_condition="tol", iter=3, friction_model="colebrook",
                    mode="all", transient=False, nonlinear_method="automatic", tol_p=1e-4,
                    tol_v=1e-4, ambient_temperature=298.15)

<img src="pics/heat_network_tutorial_example.png">

The general results for the junctions and pipes can still be accessed as follows:

In [ ]:
net.res_junction
net.res_pipe

To get the internal results regarding the division of the pipes into sections, use the following function:

In [ ]:
pipe_1_results = Pipe.get_internal_results(net, [0])

Here the results of Pipe 1 ([0]) are accessed. In general these include three matrices with the values of PINIT, VINIT and TINIT.

<img src="pics/heat_network_tutorial_pipe_results.png">

The internal results of the pipe can also be accessed separately, as shown here for Pipe 1:

In [ ]:
pipe_1_results["PINIT"]
pipe_1_results["TINIT"]
pipe_1_results["VINIT"]

For the pressures PINIT and the temperatures TINIT of the internal nodes of a pipe are *sections-1* values. For Pipe 1 they are 4, since *sections* is equal to 5 in this case. The internal flow velocities VINIT for each individual pipe section are therefore for *sections* pieces. The following figure illustrates this again, where the pipe is divided into *sections* equidistant segments and the orange lines represent the internal nodes:

<img src="pics/heat_network_tutorial_pipe_sections.png"> 

